In [1]:
def changes_from_press(stock_data, press_date, period):
    next_day = timedelta(days=1)
    time_after_release = timedelta(days=period)
    

    next_trading_day = press_date
    day_1_price = stock_data[stock_data['Date'] == press_date]['Open'].values

    while not day_1_price:
        next_trading_day = next_trading_day + next_day

        day_1_price = stock_data[stock_data['Date'] == (next_trading_day)]['Open'].values

    next_trading_day = next_trading_day + time_after_release
    day_2_price = stock_data[stock_data['Date'] == (next_trading_day)]['Close'].values

    while not day_2_price:
        next_trading_day = next_trading_day + next_day

        day_2_price = stock_data[stock_data['Date'] == (next_trading_day + next_day)]['Close'].values


    pct_change = ((day_2_price - day_1_price) / day_2_price)*100
    pct_change = pct_change[0]
    return pct_change

def get_df(ticker):
    
    stock_data = yf.Ticker(ticker)

    # get historical market data
    stock_hist = stock_data.history(period="max")

    stock_hist.reset_index(inplace=True)

    stock_hist['Date'] = pd.to_datetime(stock_hist['Date']).dt.date

    stock_hist['Pct_Close'] = stock_hist['Close'].pct_change()*100

    return stock_hist

In [5]:
# shell only work to 2019 need to parse pdf for earlier years


import requests
from bs4 import BeautifulSoup
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import pandas as pd
import re
from selenium import webdriver
import json

data = []
page_num = 0

ticker = 'shel'

stock_hist = get_df(ticker)


while True:
    
    data = []
        
    driver = webdriver.Chrome()

    driver.get("https://www.shell.com/media/news-and-media-releases.html")
    
    page = driver.page_source
    
    driver.quit()
    soup = BeautifulSoup(page, 'lxml')
    
    articles = soup.find_all('div', attrs={'data-name':'PressRelease'})
    
    for article in articles:
        press_outer_span = article.find('span', class_='space-x-1')

        press_inner_span = press_outer_span.findChildren("span" , recursive=False)
        
        press = press_inner_span[0].contents[0]
        
        date_outer_div = article.find('div')
        date = date_outer_div.find('p').contents[0]
        
        date = date.replace(',', '')
        
        date = datetime.strptime(date, '%b %d %Y').date()
        pct_change = changes_from_press(stock_hist, date, 1)
                
        data.append([date, press, pct_change])
    break

data = pd.DataFrame(data, columns=['date', 'press title', '1d change'])

dates_str = [date.strftime("%m/%d/%Y") for date in data['date'].tolist()]

dict_data = {
    'ticker': ticker,
    
    'data': {
        'dates':dates_str,
        'title': data['press title'].tolist(),
        'priceChange': data['1d change'].tolist(),
    },
}

with open(f'./data/{ticker}.json', 'w', encoding='utf-8') as f:
    json.dump(dict_data, f, ensure_ascii=False, indent=4)

C:\Users\Aidan\AppData\Local\Temp\ipykernel_6324\1447083126.py:17: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  while not day_2_price:
C:\Users\Aidan\AppData\Local\Temp\ipykernel_6324\1447083126.py:9: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  while not day_1_price:
